In [21]:
!pip install pyproj folium geojson flopy geopandas torch torch_geometric

In [22]:
import os
import ast
import traceback

# 2. 데이터 처리 및 분석 라이브러리
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon 

# 3. 수리/지하수 모델링 라이브러리 (flopy)
import flopy
from flopy.modpath import ParticleData, ParticleGroup, Modpath7Sim
from flopy.utils import PathlineFile 

# 4. 머신러닝/GNN 라이브러리 (scikit-learn, PyTorch)
import torch
from torch_geometric.data import Data
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

# 5. 지도 시각화 및 기타 유틸리티 라이브러리
import folium
from folium.plugins import MarkerCluster
from IPython.display import display
import matplotlib.pyplot as plt
from pyproj import Transformer

# [TASK 2.1] 조기경보 시스템 구축

In [23]:
# 1. 파일 경로 설정
events_path = "./data/analysis_summary_pollution_events.csv"
standards_path = "./data/groundwater_quality_standards2.csv"
save_path = "./data/early_warning_results.csv"


# 2. 기준표 불러오기
def parse_threshold(val):
    if isinstance(val, str):
        val = val.strip()
        if "불검출" in val or "미검출" in val:
            return 0.0
        elif "<=" in val:
            try:
                return float(val.replace("≤", "").replace("<=", "").strip())
            except:
                return None
        else:
            try:
                return float(val)
            except:
                return None
    return None

standard_df = pd.read_csv(standards_path, encoding='utf-8-sig')
standard_df['기준값(숫자)'] = standard_df['기준값'].apply(parse_threshold)
standard_df = standard_df[standard_df['event_항목'].notnull()]  # 이벤트 기준명 있는 것만

# 딕셔너리 생성: {이벤트파일 기준 항목명: 기준값}
standard_dict = dict(zip(standard_df['event_항목'], standard_df['기준값(숫자)']))

# 3. 이벤트 파일 불러오기
events_df = pd.read_csv(events_path)

# 4. 기준표 항목 중 이벤트 파일에 실제 존재하는 항목만 필터링
available_columns = events_df.columns
target_columns = [col for col in standard_dict.keys() if col in available_columns]

# 5. 초과 여부 계산
for col in target_columns:
    threshold = standard_dict[col]
    events_df[f"{col}_초과"] = events_df[col] > threshold

# 6. 초과 항목 수 및 항목명 리스트 계산
exceed_columns = [f"{col}_초과" for col in target_columns]
events_df["초과항목수"] = events_df[exceed_columns].sum(axis=1)

def get_exceed_list(row):
    return [col for col in target_columns if row[f"{col}_초과"]]

events_df["초과항목목록"] = events_df.apply(get_exceed_list, axis=1)


# 7. 최종 조기경보 결과 저장
save_cols = [
    "공번호", "조사시기", "시군구명", "TM_X_5186", "TM_Y_5186",
    "초과항목수", "초과항목목록"
] + exceed_columns

events_df[save_cols].to_csv(save_path, index=False, encoding="utf-8-sig")
print("✅ 조기경보 결과 저장 완료:", save_path)

✅ 조기경보 결과 저장 완료: ./data/early_warning_results.csv


## FOLIUM 지도화

In [24]:
df = pd.read_csv("./data/early_warning_results.csv")

# -------------------------------------------------------------
# 3. 좌표 변환기 생성: EPSG:5186(TMK) → EPSG:4326(WGS84)
transformer = Transformer.from_crs("EPSG:5186", "EPSG:4326", always_xy=True)

# 변환된 좌표 저장할 컬럼 생성
def convert_coords(row):
    try:
        lon, lat = transformer.transform(row["TM_X_5186"], row["TM_Y_5186"])
        return pd.Series([lat, lon])
    except:
        return pd.Series([None, None])

df[['lat', 'lon']] = df.apply(convert_coords, axis=1)

# -------------------------------------------------------------
# 4. folium 지도 생성
m = folium.Map(location=[36.5, 127.8], zoom_start=8)
marker_cluster = MarkerCluster().add_to(m)

# -------------------------------------------------------------
# 5. 마커 추가 (초과항목목록 파싱 robust)
for idx, row in df.iterrows():
    try:
        if pd.notnull(row['lat']) and pd.notnull(row['lon']):
            # 리스트 문자열 safely eval
            try:
                exceed_list = ast.literal_eval(row['초과항목목록']) if isinstance(row['초과항목목록'], str) else []
            except:
                exceed_list = []
            popup_text = f"""
            <b>공번호:</b> {row['공번호']}<br>
            <b>시군구:</b> {row['시군구명']}<br>
            <b>조사시기:</b> {row['조사시기']}<br>
            <b>초과항목수:</b> {row['초과항목수']}<br>
            <b>초과항목:</b> {', '.join(exceed_list)}
            """
            color = 'red' if row['초과항목수'] >= 2 else 'orange' if row['초과항목수'] == 1 else 'blue'
            folium.CircleMarker(
                location=[row['lat'], row['lon']],
                radius=6 + row['초과항목수'] * 1.5,
                color=color,
                fill=True,
                fill_opacity=0.7,
                popup=folium.Popup(popup_text, max_width=300)
            ).add_to(marker_cluster)
    except:
        continue

# -------------------------------------------------------------
# 6. 지도 렌더링

m


# [TASK 2.2] 오염 경로 역추적 시스템 설계

## 1. 초과항목수 > 0인 관측소 필터링 및 좌표 변환

In [25]:
# 1. 초과항목수 > 0인 데이터만 필터링
events_df = events_df[events_df['초과항목수'] > 0].copy()

# 2. MODFLOW 모델링에 사용할 데이터 준비
# 좌표 변환 (TM -> WGS84)
from pyproj import Transformer
transformer = Transformer.from_crs("EPSG:5186", "EPSG:4326", always_xy=True)

def convert_coords(row):
    try:
        lon, lat = transformer.transform(row["TM_X_5186"], row["TM_Y_5186"])
        return pd.Series([lat, lon])
    except:
        return pd.Series([None, None])

# 좌표 변환 실행
events_df[['lat', 'lon']] = events_df.apply(convert_coords, axis=1)

# 3. 필터링된 이벤트의 초과항목수와 좌표 확인
events_df[['공번호', '초과항목수', 'lat', 'lon']].head()

,공번호,초과항목수,lat,lon
0,BGE02,1,35.923050,128.620603
1,DGE02,3,35.903831,128.714428
2,DSD1335,1,35.688608,128.356353
3,DSD2262,1,35.860291,128.408070
4,DSE02,1,35.874666,128.466367


In [26]:
# 초과항목수 별 갯수 확인
exceed_count = events_df['초과항목수'].value_counts().sort_index(ascending=False)

# 결과 출력
print(" 초과항목수별 갯수:")
for count, num in exceed_count.items():
    print(f"초과항목수 {count}개: {num}건")

 초과항목수별 갯수:
초과항목수 4개: 1건
초과항목수 3개: 9건
초과항목수 2개: 67건
초과항목수 1개: 585건


## 2. MODFLOW 격자 기반 수리해석 환경 세팅

In [27]:
import os
import zipfile
import flopy

print("="*60)
print("MODFLOW 모델 자동 실행 (최종 수정 버전)")
print("="*60)

# --- 1. 경로 설정 ---
project_root = os.getcwd()
print(f"✅ 프로젝트 경로: '{project_root}'")

data_dir = os.path.join(project_root, 'data')
program_dir = os.path.join(project_root, '1_프로그램')
workspace_dir = os.path.join(project_root, '2_모델_작업공간')
mf6_zip_path = os.path.join(data_dir, 'mf6.6.2_win64.Zip.zip')

# --- ✨✨✨✨✨ 진짜 최종 수정 포인트! ✨✨✨✨✨ ---
# 압축 파일 내에 상위 폴더가 없으므로 경로에서 제거합니다.
mf6_exe_path = os.path.join(program_dir, 'bin', 'mf6.exe') 
# --------------------------------------------------

# --- 2. 자동 압축 해제 ---
def unzip_if_needed(zip_path, extract_to, target_check):
    os.makedirs(extract_to, exist_ok=True)
    if os.path.exists(target_check):
        print(f"✅ 프로그램이 이미 준비되어 있습니다. 압축 해제를 건너뜁니다.")
        return
    print(f"⏳ '{os.path.basename(zip_path)}'의 압축을 해제합니다...")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"✅ 압축 해제 완료 -> '{extract_to}'")
    except Exception as e:
        print(f"🚨 압축 해제 실패: {e}")
        exit()

unzip_if_needed(mf6_zip_path, program_dir, mf6_exe_path)

# 압축 해제 후 파일이 준비되었는지 최종 확인
if not os.path.exists(mf6_exe_path):
    print(f"🚨 치명적 오류: 압축을 풀었음에도 '{mf6_exe_path}' 경로에 파일이 없습니다. zip 파일 내용을 확인해주세요.")
    exit()

# --- 3. MODFLOW 시뮬레이션 설정 및 실행 ---
print("\n--- MODFLOW 시뮬레이션 시작 ---")
sim = flopy.mf6.MFSimulation(sim_name="testmodel", exe_name=mf6_exe_path, sim_ws=workspace_dir)
tdis = flopy.mf6.ModflowTdis(sim, time_units="DAYS", nper=1, perioddata=[(1.0, 1, 1.0)])
gwf = flopy.mf6.ModflowGwf(sim, modelname="gwfmodel", save_flows=True)
ims = flopy.mf6.ModflowIms(sim, print_option="SUMMARY", complexity="SIMPLE")
sim.register_ims_package(ims, [gwf.name])
dis = flopy.mf6.ModflowGwfdis(gwf, nlay=1, nrow=100, ncol=100, delr=100, delc=100, top=10, botm=0)
ic = flopy.mf6.ModflowGwfic(gwf, strt=10)
npf = flopy.mf6.ModflowGwfnpf(gwf, icelltype=1, k=10)
sto = flopy.mf6.ModflowGwfsto(gwf, iconvert=1, ss=1e-4, sy=0.3)
oc = flopy.mf6.ModflowGwfoc(
    gwf, budget_filerecord="gwfmodel.cbc", head_filerecord="gwfmodel.hds",
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
    printrecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
)

sim.write_simulation()
success, buff = sim.run_simulation()

if success:
    print("\n\n🎉🎉🎉 시뮬레이션 최종 성공! 🎉🎉🎉")
    print(f"결과 파일들은 '{workspace_dir}' 폴더에 저장되었습니다.")
else:
    print("\n\n🚨🚨🚨 시뮬레이션 실행 단계에서 실패... 🚨🚨🚨")
    for line in buff:
        print(line)

MODFLOW 모델 자동 실행 (최종 수정 버전)
✅ 프로젝트 경로: 'd:\underwater_data\final'
✅ 프로그램이 이미 준비되어 있습니다. 압축 해제를 건너뜁니다.

--- MODFLOW 시뮬레이션 시작 ---
writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...
  writing model gwfmodel...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package sto...
    writing package oc...
FloPy is using the following executable to run the model: ..\1_프로그램\bin\mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                            VERSION 6.6.2 05/12/2025

   MODFLOW 6 compiled May 12 2025 12:42:18 with Intel(R) Fortran Intel(R) 64
   Compiler Classic for applications running on Intel(R) 64, Version 2021.7.0
                             Build 20220726_000000

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the soft

In [28]:
import os
import flopy

try:
    project_root = os.path.dirname(os.path.abspath(__file__))
    print("✅ (.py 파일 실행 모드) 스크립트 파일 기준으로 경로를 설정합니다.")

except NameError:
    project_root = os.getcwd()
    print("✅ (Jupyter/IPython 모드) 현재 작업 폴더를 기준으로 경로를 설정합니다.")

print(f"   - 프로젝트 경로: {project_root}")

# 저장된 폴더의 경로
exe_path = os.path.join(project_root, '1_프로그램', 'bin', 'mf6.exe')
workspace_path = os.path.join(project_root, '2_모델_작업공간')

sim = flopy.mf6.MFSimulation(
    sim_name="testmodel",
    exe_name=exe_path,
    sim_ws=workspace_path
)


tdis = flopy.mf6.ModflowTdis(sim, time_units="DAYS", nper=1, perioddata=[(1.0, 1, 1.0)])

# 모델(domain) 생성
gwf = flopy.mf6.ModflowGwf(sim, modelname="gwfmodel", save_flows=True)
ims = flopy.mf6.ModflowIms(sim, print_option="SUMMARY", complexity="SIMPLE")
sim.register_ims_package(ims, [gwf.name])

dis = flopy.mf6.ModflowGwfdis(gwf, nlay=1, nrow=100, ncol=100, delr=100, delc=100, top=10, botm=0)
ic = flopy.mf6.ModflowGwfic(gwf, strt=10)
npf = flopy.mf6.ModflowGwfnpf(gwf, icelltype=1, k=10)
sto = flopy.mf6.ModflowGwfsto(gwf, iconvert=1, ss=1e-4, sy=0.3)
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    budget_filerecord="gwfmodel.cbc",
    head_filerecord="gwfmodel.hds",
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
    printrecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
)

# 폴더가 없으면 새로 생성해줍니다.
os.makedirs(workspace_path, exist_ok=True) 

sim.write_simulation()
success, buff = sim.run_simulation()

if success:
    print("\n🎉 MODFLOW6 실행 결과: 성공")
    print(f"   결과 파일은 '{workspace_path}' 폴더를 확인하세요.")
else:
    print("\n🚨 MODFLOW6 실행 결과: 실패")

✅ (Jupyter/IPython 모드) 현재 작업 폴더를 기준으로 경로를 설정합니다.
   - 프로젝트 경로: d:\underwater_data\final
writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...
  writing model gwfmodel...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package sto...
    writing package oc...
FloPy is using the following executable to run the model: ..\1_프로그램\bin\mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                            VERSION 6.6.2 05/12/2025

   MODFLOW 6 compiled May 12 2025 12:42:18 with Intel(R) Fortran Intel(R) 64
   Compiler Classic for applications running on Intel(R) 64, Version 2021.7.0
                             Build 20220726_000000

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the software has been subjected to rigorous 
rev

## 3. 조기경보 파일에서 이상 감지 지점 추출

In [29]:
import pandas as pd

# 1. 조기경보 파일 불러오기
early_warning_path = "./data/early_warning_results.csv"
ew_df = pd.read_csv(early_warning_path, encoding='utf-8-sig')

# 2. 초과항목수 가장 많은 한 지점(예시) 추출
target = ew_df.loc[ew_df['초과항목수'].idxmax()]
print("관측소 조기경보 위치:")
print("공번호:", target['공번호'])
print("TM_X_5186:", target['TM_X_5186'])
print("TM_Y_5186:", target['TM_Y_5186'])

# 3. 이 좌표를 기억! (다음 단계에서 격자 변환에 사용)
tmx = target["TM_X_5186"]
tmy = target["TM_Y_5186"]

관측소 조기경보 위치:
공번호: YIW95003
TM_X_5186: 214670.9973245879
TM_Y_5186: 501864.0005511716


## 4. TM 좌표 → MODFLOW 격자(row, col) 변환

In [30]:
# 예시로 일단 진행
x0 = 214000    # domain 좌상단 TM_X_5186 값 (네 모델 기준 최소)
y0 = 501000    # domain 좌상단 TM_Y_5186 값 (네 모델 기준 최소)
delr = 100     # 셀 크기 (m)
delc = 100     # 셀 크기 (m)

def coord_to_index(x, y, x0, y0, delr, delc):
    col = int((x - x0) // delr)
    row = int((y - y0) // delc)
    return row, col

# 네 관측소 좌표
tmx = 214670.9973245879
tmy = 501864.0005511716

row, col = coord_to_index(tmx, tmy, x0, y0, delr, delc)
print(f"MODFLOW 격자 위치: layer=0, row={row}, col={col}")


MODFLOW 격자 위치: layer=0, row=8, col=6


## 5. MODPATH7 파티클 그룹 선언 및 시뮬레이션 실행

In [31]:
import os
import flopy

try:
    project_root = os.path.dirname(os.path.abspath(__file__))
    print("✅ (.py 파일 실행 모드) 스크립트 파일 기준으로 경로를 설정합니다.")
except NameError:
    project_root = os.getcwd()
    print("✅ (Jupyter/IPython 모드) 현재 작업 폴더를 기준으로 경로를 설정합니다.")

# 1. 시뮬레이션 결과가 저장된 '작업 폴더' 경로
workspace = os.path.join(project_root, '2_모델_작업공간')

# 2. MODFLOW 6 와 MODPATH 7 실행 프로그램 경로
mf6_exe = os.path.join(project_root, '1_프로그램', 'bin', 'mf6.exe')
mp7_exe = os.path.join(project_root, '1_프로그램', 'bin', 'mpath7.exe') # MODPATH 경로 추가

print(f"   - 작업 폴더 경로: {workspace}")


# 기존에 성공적으로 실행된 MFSimulation 결과
print("\n--- 기존 MODFLOW 6 모델을 불러옵니다 ---")
sim = flopy.mf6.MFSimulation.load(
    sim_name="testmodel",      # 저장했던 시뮬레이션 이름
    sim_ws=workspace,          # 자동으로 찾은 작업 폴더 경로
    exe_name=mf6_exe           # 자동으로 찾은 mf6.exe 경로
)
gwf = sim.get_model("gwfmodel")
print("✅ 모델 로드 성공!")


# MODPATH7 인스턴스 생성 (MODFLOW 모델 객체 넘겨주기)
print("\n--- MODPATH 7 분석을 설정합니다 ---")
mp = flopy.modpath.Modpath7(
    modelname="modpath7_model",   # 새로운 MODPATH 모델의 이름
    flowmodel=gwf,                # 유동 해석에 사용할 MODFLOW 모델 지정
    exe_name=mp7_exe,             # 자동으로 찾은 mpath7.exe 경로
    model_ws=workspace            # MODPATH 결과도 같은 작업 폴더에 저장
)
print("✅ MODPATH 7 설정 완료!")

✅ (Jupyter/IPython 모드) 현재 작업 폴더를 기준으로 경로를 설정합니다.
   - 작업 폴더 경로: d:\underwater_data\final\2_모델_작업공간

--- 기존 MODFLOW 6 모델을 불러옵니다 ---
loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package ic...
    loading package npf...
    loading package sto...
    loading package oc...
  loading solution package gwfmodel...
✅ 모델 로드 성공!

--- MODPATH 7 분석을 설정합니다 ---
✅ MODPATH 7 설정 완료!


## 6. MODPATH 결과 파일에서 위치 추출 및 정제

In [32]:
from flopy.modpath import ParticleData, ParticleGroup, Modpath7Bas, Modpath7Sim

# 1. 입자 위치 정의
row = 8
col = 6

particledata = ParticleData(
    [[0, row, col]],       # layer, row, col
    structured=True,
    particleids=[1],
    localx=[0.5],
    localy=[0.5],
    localz=[0.5],
    timeoffset=[0.0]
)

pg = ParticleGroup(
    particlegroupname="group1",
    particledata=particledata
)

# 2. 기본 설정
mpbas = Modpath7Bas(mp, porosity=0.3)

# ❗ 여기를 수정: terminate → stop_at
mpsim = Modpath7Sim(
    mp,
    simulationtype='pathline',           # 또는 'endpoint'
    trackingdirection='backward',        # 역추적
    weaksinkoption='stop_at',            # 유효한 옵션
    weaksourceoption='stop_at',
    budgetoutputoption='summary',
    particlegroups=[pg]
)

# 3. 실행
mp.write_input()
success, buff = mp.run_model()
print("✅ MODPATH7 실행 성공 여부:", success)


import flopy.utils.binaryfile as bf
headobj = bf.HeadFile(workspace + "/gwfmodel.hds")
head = headobj.get_data(totim=1.0)

# 입자 위치 head 값 확인
print("입자 위치의 수위:", head[0, row, col])


FloPy is using the following executable to run the model: ..\1_프로그램\bin\mpath7.exe

MODPATH Version 7.2.001   
Program compiled Dec 22 2017 11:11:36 with IFORT compiler (ver. 16.0.0)         
 
 
Run particle tracking simulation ...
Processing Time Step     1 Period     1.  Time =  1.00000E+00  Steady-state flow                                                    

Particle Summary:
         0 particles are pending release.
         0 particles remain active.
         0 particles terminated at boundary faces.
         0 particles terminated at weak sink cells.
         0 particles terminated at weak source cells.
         1 particles terminated at strong source/sink cells.
         0 particles terminated in cells with a specified zone number.
         0 particles were stranded in inactive or dry cells.
         0 particles were unreleased.
         0 particles have an unknown status.
 
Normal termination.                                                        
✅ MODPATH7 실행 성공 여부: True


## 7. MODFLOW 상대좌표 → TM → 위경도 변환

In [33]:
from flopy.utils import PathlineFile
import os
import pandas as pd

pathline_file = os.path.join(workspace, "modpath7_model.mppth")
print(f"✅ 결과 파일 경로를 확인합니다: '{pathline_file}'")

try:
    pfile = PathlineFile(pathline_file)
    print("✅ 결과 파일을 성공적으로 열었습니다.")

    all_data = pfile.get_alldata()
    
    if not all_data:
        print("⚠️ 결과 파일에 추적된 입자 경로 데이터가 없습니다.")
        points_df = pd.DataFrame()
    else:
        records = []
        for arr in all_data:
            for rec in arr:
                records.append(dict(zip(rec.dtype.names, rec)))
        
        points_df = pd.DataFrame(records)
        print("✅ 경로 데이터를 DataFrame으로 성공적으로 변환했습니다.")

    print("\n--- 변환된 데이터프레임 정보 ---")
    print("DataFrame 모양(행, 열):", points_df.shape)
    print("상위 5개 데이터:")
    print(points_df.head())
    
except FileNotFoundError:
    print(f"🚨 오류: '{pathline_file}' 경로에 파일이 없습니다! MODPATH 실행이 성공했는지, 파일 이름을 다시 확인해주세요.")
except Exception as e:
    print(f"🚨 데이터를 읽는 중 예상치 못한 오류가 발생했습니다: {e}")

✅ 결과 파일 경로를 확인합니다: 'd:\underwater_data\final\2_모델_작업공간\modpath7_model.mppth'
✅ 결과 파일을 성공적으로 열었습니다.
✅ 경로 데이터를 DataFrame으로 성공적으로 변환했습니다.

--- 변환된 데이터프레임 정보 ---
DataFrame 모양(행, 열): (2, 15)
상위 5개 데이터:
   particleid  particlegroup  sequencenumber  particleidloc  time      x  \
0           0              0               0              0   0.0  650.0   
1           0              0               0              0   0.0  650.0   

        y    z  k  node  xloc  yloc  zloc  stressperiod  timestep  
0  9150.0  5.0  0   806   0.5   0.5   0.5             1         1  
1  9150.0  5.0  0   806   0.5   0.5   0.5             1         1  


In [34]:
# 가장 과거(time 최대 or 최소, 모델 설정에 따라 다름)
# 예시: time이 작을수록 최신 → 역추적이면 time이 클수록 "상류"
final_points = points_df.sort_values("time").groupby("particleid").tail(1)
print(final_points[["particleid", "x", "y", "time"]])

   particleid      x       y  time
1           0  650.0  9150.0   0.0


## 8. 추정된 오염원 위치 Folium 지도 시각화

In [35]:
from pyproj import Transformer

x0 = 214000
y0 = 501000

transformer = Transformer.from_crs("EPSG:5186", "EPSG:4326", always_xy=True)

def tm_to_latlon(row):
    tm_x = x0 + row["x"]
    tm_y = y0 + row["y"]
    lon, lat = transformer.transform(tm_x, tm_y)
    return pd.Series([lat, lon])

final_points[["lat", "lon"]] = final_points.apply(tm_to_latlon, axis=1)


import folium

# 중심 위치 평균값
m = folium.Map(location=[final_points["lat"].mean(), final_points["lon"].mean()], zoom_start=14)

# 입자별 마커 추가
for _, row in final_points.iterrows():
    folium.Marker(
        location=[row["lat"], row["lon"]],
        popup=f"입자 {int(row['particleid'])}",
        icon=folium.Icon(color='red', icon='tint')
    ).add_to(m)

m


In [36]:
import geopandas as gpd
from shapely.geometry import Point

final_points['TM_X_5186'] = x0 + final_points['x']
final_points['TM_Y_5186'] = y0 + final_points['y']

geometry = [Point(xy) for xy in zip(final_points['TM_X_5186'], final_points['TM_Y_5186'])]

source_points_gdf = gpd.GeoDataFrame(
    final_points, 
    geometry=geometry, 
    crs="EPSG:5186"
)

print("✅ GeoDataFrame이 성공적으로 생성되었습니다.")
print(source_points_gdf.head())

✅ GeoDataFrame이 성공적으로 생성되었습니다.
   particleid  particlegroup  sequencenumber  particleidloc  time      x  \
1           0              0               0              0   0.0  650.0   

        y    z  k  node  xloc  yloc  zloc  stressperiod  timestep        lat  \
1  9150.0  5.0  0   806   0.5   0.5   0.5             1         1  37.190344   

          lon  TM_X_5186  TM_Y_5186               geometry  
1  127.164997   214650.0   510150.0  POINT (214650 510150)  


In [37]:
import os
import geopandas as gpd
from shapely.geometry import Polygon


# 저장 폴더 경로 지정하고, 폴더가 없으면 생성
data_folder = os.path.join(project_root, 'data')
os.makedirs(data_folder, exist_ok=True)


# 가장 작은 다각형(오염원 추정 영역) 계산
probable_source_area_polygon = source_points_gdf.unary_union.convex_hull

# 다각형(Polygon) →  GeoDataFrame
probable_source_area_gdf = gpd.GeoDataFrame(
    crs="EPSG:5186",
    geometry=[probable_source_area_polygon]
)

# 파일 이름을 data 폴더 경로와 합쳐줍니다.
output_filename = os.path.join(data_folder, 'probable_source_area.geojson')

# GeoDataFrame →  GeoJSON 파일
probable_source_area_gdf.to_file(output_filename, driver='GeoJSON', encoding='utf-8')


print(f"\n============================================================")
print(f"✅✅✅ 최종 결과물인 '{output_filename}' 파일이 성공적으로 저장되었습니다!")


✅✅✅ 최종 결과물인 'd:\underwater_data\final\data\probable_source_area.geojson' 파일이 성공적으로 저장되었습니다!


C:\Users\user\AppData\Local\Temp\ipykernel_3604\4058980383.py:12: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  probable_source_area_polygon = source_points_gdf.unary_union.convex_hull


# Task 2.3: Graph Neural Network 기반 오염원 유형 분류 (Source-Type Classification via GNN)

In [38]:
probable_source_area = "./data/probable_source_area.geojson"
gims_master_data = "./data/gims_master_data2.csv"
feature_matrix = "./data/feature_matrix_by_sigungu.csv"

## **분석 대상 선정: 가장 가까운 관측소 탐색**
* 오염원 후보 영역('사건 현장') 내부에 포함된 관측소가 없을 경우를 대비하여,
* 영역의 중심점에서 '가장 가까운 5개 관측소'를 분석의 주인공(노드)으로 선정

In [39]:
probable_source_area = gpd.read_file("./data/probable_source_area.geojson")
gims_master_data = pd.read_csv("./data/gims_master_data2.csv", encoding='utf-8')
feature_matrix = pd.read_csv("./data/feature_matrix_by_sigungu.csv", encoding='utf-8')

# 1. '사건 현장'의 중심점 계산
source_centroid = probable_source_area.unary_union.centroid

# 2. 전체 관측소 데이터를 지도 데이터로 변환
gdf_gims = gpd.GeoDataFrame(
    gims_master_data,
    geometry=gpd.points_from_xy(gims_master_data.TM_X_5186, gims_master_data.TM_Y_5186),
    crs="EPSG:5186"
)

# 3. 중심점과의 거리를 계산하고, 가장 가까운 5개 관측소 선택
gdf_gims['distance_to_source'] = gdf_gims.geometry.distance(source_centroid)
nodes_in_area = gdf_gims.sort_values(by='distance_to_source').head(5)

print("✅ '사건 현장'에서 가장 가까운 5개 핵심 관측소는 다음과 같습니다:")
display(nodes_in_area[['공번호', '시군구명', 'distance_to_source']].rename(columns={
    '공번호': '관측소 번호', '시군구명': '소재지', 'distance_to_source': '사건 현장과의 거리(m)'
}))

✅ '사건 현장'에서 가장 가까운 5개 핵심 관측소는 다음과 같습니다:


C:\Users\user\AppData\Local\Temp\ipykernel_3604\3061848787.py:6: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  source_centroid = probable_source_area.unary_union.centroid


,관측소 번호,소재지,사건 현장과의 거리(m)
305,YIW25034,경기도 용인시,2397.658062
284,YIW00518,경기도 용인시,5690.089432
306,YIW11861,경기도 용인시,7187.638556
285,YIW00444,경기도 용인시,7277.401920
288,YIW25003,경기도 용인시,8130.565667


In [40]:
# 지역이름 표준화 작업 진행 
nodes_in_area['join_key'] = nodes_in_area['시군구명'].str.replace(' ', '_').str.replace(r'시|군|구$', '', regex=True)
feature_matrix['join_key'] = feature_matrix['지역'].str.replace(' ', '_').str.replace(r'시|군|구$', '', regex=True)

# 통일된 'join_key'를 기준으로 두 데이터 결합
nodes_with_features = pd.merge(nodes_in_area, feature_matrix, on='join_key', how='inner')

# 불필요한 컬럼 정리 및 안정성 확보
nodes_with_features.drop(columns=['join_key', '지역'], inplace=True, errors='ignore')
nodes_with_features.fillna(0, inplace=True)

print(f"✅ 데이터 결합 성공! 총 {len(nodes_with_features)}개의 노드(관측소) 정보가 준비되었습니다.")


✅ 데이터 결합 성공! 총 5개의 노드(관측소) 정보가 준비되었습니다.


## **GNN 그래프 생성**
* 분석 대상 관측소들을 '점(Node)'으로, 각 관측소의 지역적/환경적 특징을 '점의 정보(Feature)'로,
* 그리고 관측소 간의 물리적 거리를 '관계선(Edge)'으로 정의하여 GNN이 이해할 수 있는 그래프 데이터를 생성

In [41]:
# 1. GNN의 입력으로 사용할 특징(Feature) 컬럼 목록 생성
feature_cols = [col for col in feature_matrix.columns if col not in ['지역', 'join_key']]
node_features = nodes_with_features[feature_cols].to_numpy(dtype=np.float32)

# 2. 노드 위치 정보 추출 및 이웃 관계 정의 (KNN)
node_positions = nodes_with_features[['TM_X_5186', 'TM_Y_5186']].to_numpy(dtype=np.float32)
knn = NearestNeighbors(n_neighbors=min(4, len(nodes_with_features)), metric='euclidean').fit(node_positions)
adj_matrix = knn.kneighbors_graph(mode='connectivity').toarray()

# 3. 최종 GNN 데이터 객체 생성
edge_index = torch.tensor(np.array(np.nonzero(adj_matrix)), dtype=torch.long)
x = torch.tensor(node_features, dtype=torch.float)
graph_data = Data(x=x, edge_index=edge_index)

print("✅ GNN 그래프 생성이 완료되었습니다.")
print(f"   - 노드 수: {graph_data.num_nodes}, 엣지 수: {graph_data.num_edges}, 노드 특징 수: {graph_data.num_node_features}")

# [상세 설명 추가] 생성된 GNN 그래프의 구조를 자세히 설명합니다.
print("\n--- [4단계 결과 상세] ---\n생성된 GNN 그래프의 구조는 다음과 같습니다:")
print(f"   - 노드(점)의 총 개수: {graph_data.num_nodes}개 (우리가 분석하는 핵심 관측소의 수)")
print(f"   - 각 노드가 가진 특징(정보)의 수: {graph_data.num_node_features}개 (지역별 오염 프로파일의 종류)")
print(f"   - 엣지(관계선)의 총 개수: {graph_data.num_edges}개 (관측소 간의 연결 관계의 수)")
print("   - 상세 연결 관계:")
# edge_index를 사람이 이해하기 쉬운 형태로 출력
for i in range(graph_data.num_edges):
    source_node = graph_data.edge_index[0, i]
    target_node = graph_data.edge_index[1, i]
    print(f"     * 관측소 {source_node} 와(과) 관측소 {target_node} 가 연결되었습니다.")
    print("-" * 50)

✅ GNN 그래프 생성이 완료되었습니다.
   - 노드 수: 5, 엣지 수: 20, 노드 특징 수: 9

--- [4단계 결과 상세] ---
생성된 GNN 그래프의 구조는 다음과 같습니다:
   - 노드(점)의 총 개수: 5개 (우리가 분석하는 핵심 관측소의 수)
   - 각 노드가 가진 특징(정보)의 수: 9개 (지역별 오염 프로파일의 종류)
   - 엣지(관계선)의 총 개수: 20개 (관측소 간의 연결 관계의 수)
   - 상세 연결 관계:
     * 관측소 0 와(과) 관측소 1 가 연결되었습니다.
--------------------------------------------------
     * 관측소 0 와(과) 관측소 2 가 연결되었습니다.
--------------------------------------------------
     * 관측소 0 와(과) 관측소 3 가 연결되었습니다.
--------------------------------------------------
     * 관측소 0 와(과) 관측소 4 가 연결되었습니다.
--------------------------------------------------
     * 관측소 1 와(과) 관측소 0 가 연결되었습니다.
--------------------------------------------------
     * 관측소 1 와(과) 관측소 2 가 연결되었습니다.
--------------------------------------------------
     * 관측소 1 와(과) 관측소 3 가 연결되었습니다.
--------------------------------------------------
     * 관측소 1 와(과) 관측소 4 가 연결되었습니다.
--------------------------------------------------
     * 관측소 2 와(과) 관측소 0 가 연결되었습니다.
--------------------------

## 💧 오염원 유형 추론
* 생성된 GNN 그래프의 종합적인 특징(Embedding)과, 전형적인 '축산 지역' 및 '산업 지역'의 특징을 비교하여
* '코사인 유사도'를 계산하고, 최종적으로 오염원의 유형 확률을 추론

In [42]:
graph_embedding = graph_data.x.mean(axis=0).numpy().reshape(1, -1)
cattle_archetype = feature_matrix.iloc[feature_matrix['축산계_고위험배출량'].idxmax()][feature_cols].to_numpy().reshape(1, -1)
industry_archetype = feature_matrix.iloc[feature_matrix['산업계_총폐수방류량'].idxmax()][feature_cols].to_numpy().reshape(1, -1)

similarity_cattle = cosine_similarity(graph_embedding, cattle_archetype)[0, 0]
similarity_industry = cosine_similarity(graph_embedding, industry_archetype)[0, 0]

print("="*20 + " [ GNN 기반 오염원 유형 최종 추론 결과 ] " + "="*20)
print(f"\n[분석] '사건 현장'과 '대표 축산 지역'의 유사도: {similarity_cattle:.2%}")
print(f"[분석] '사건 현장'과 '대표 산업 지역'의 유사도: {similarity_industry:.2%}")

if similarity_cattle + similarity_industry == 0:
    print("\n[최종 결론] 두 유형과의 유사도가 모두 0으로, 판별이 불가능합니다.")
else:
    if similarity_cattle > similarity_industry:
        winner = "축산계 오염"
        probability = similarity_cattle / (similarity_cattle + similarity_industry)
    else:
        winner = "산업계 오염"
        probability = similarity_industry / (similarity_cattle + similarity_industry)
    print(f"\n[최종 결론] 이번 오염은 '{winner}'일 확률이 약 {probability:.2%}로 강력하게 추정됩니다.")
print("=" * 78)


==================== [ GNN 기반 오염원 유형 최종 추론 결과 ] ====================

[분석] '사건 현장'과 '대표 축산 지역'의 유사도: 7.02%
[분석] '사건 현장'과 '대표 산업 지역'의 유사도: 0.00%

[최종 결론] 이번 오염은 '축산계 오염'일 확률이 약 100.00%로 강력하게 추정됩니다.
